<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">

# Procesamiento de Lenguaje Natural
## Desafío 3: Modelo de Lenguaje con Tokenización por Caracteres

**Autor:** Carlos Espinola  
**Fecha:** Diciembre 2025

---
## Objetivos del Desafío

### Consigna
1. **Seleccionar un corpus de texto** sobre el cual entrenar el modelo de lenguaje
2. **Pre-procesamiento**: tokenizar el corpus, estructurar el dataset y separar datos de entrenamiento y validación
3. **Proponer arquitecturas RNN**: implementar modelos basados en unidades recurrentes (SimpleRNN, LSTM, GRU)
4. **Generación de secuencias** con diferentes estrategias:
   - Greedy Search
   - Beam Search Determinístico
   - Beam Search Estocástico (analizando el efecto de la temperatura)

### Sugerencias
- Guiarse por el descenso de la **perplejidad** en validación para finalizar el entrenamiento
- Explorar: SimpleRNN (celda de Elman), LSTM y GRU
- `RMSprop` es el optimizador recomendado para buena convergencia


In [ ]:
# =============================================================================
# 1. IMPORTACIÓN DE LIBRERÍAS
# =============================================================================

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import urllib.request
import bs4 as bs

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset

# Configuración de estilo para gráficos
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# Configuración del dispositivo (GPU si está disponible)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️ Dispositivo: {device}")
print(f"🔧 PyTorch version: {torch.__version__}")

---
## 2. Selección y Descarga del Corpus

Utilizaremos el libro **"La Vuelta al Mundo en 80 Días"** de Julio Verne como corpus de entrenamiento. Este texto en español nos proporciona un corpus extenso y de calidad literaria para entrenar nuestro modelo de lenguaje a nivel de caracteres.

In [ ]:
# Descargar el libro desde textos.info
url = 'https://www.textos.info/julio-verne/la-vuelta-al-mundo-en-80-dias/ebook'
raw_html = urllib.request.urlopen(url)
raw_html = raw_html.read()

# Parsear el HTML con BeautifulSoup
article_html = bs.BeautifulSoup(raw_html, 'lxml')

# Extraer todos los párrafos
article_paragraphs = article_html.find_all('p')

# Concatenar el texto de todos los párrafos
corpus = ''
for para in article_paragraphs:
    corpus += para.text + ' '

# Convertir a minúsculas para normalizar
corpus = corpus.lower()

print(f"📚 Longitud total del corpus: {len(corpus):,} caracteres")
print(f"\n📖 Primeros 500 caracteres del corpus:")
print("-" * 50)
print(corpus[:500])

In [ ]:
# Análisis de distribución de caracteres en el corpus
char_counts = Counter(corpus)
most_common = char_counts.most_common(20)

fig, ax = plt.subplots(figsize=(14, 5))
chars, counts = zip(*most_common)
chars_display = [repr(c) if c in [' ', '\n', '\t'] else c for c in chars]
bars = ax.bar(chars_display, counts, color='steelblue', edgecolor='navy', alpha=0.8)
ax.set_xlabel('Caracter', fontsize=12)
ax.set_ylabel('Frecuencia', fontsize=12)
ax.set_title('📊 Distribución de los 20 caracteres más frecuentes en el corpus', fontsize=14)
plt.xticks(rotation=45, fontsize=11)

# Añadir valores encima de las barras
for bar, count in zip(bars, counts):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1000, 
            f'{count:,}', ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.show()

In [ ]:
---
## 3. Tokenización por Caracteres

En un modelo de lenguaje por caracteres, cada carácter único del corpus representa un token. Esto nos permite:
- **Vocabulario pequeño**: Típicamente 50-100 caracteres vs miles de palabras
- **Flexibilidad**: Puede manejar cualquier palabra, incluso neologismos
- **Captura morfológica**: Aprende patrones dentro de las palabras

# Crear vocabulario de caracteres únicos (ordenado para reproducibilidad)
chars_vocab = sorted(set(corpus))
vocab_size = len(chars_vocab)

print(f"📝 Tamaño del vocabulario: {vocab_size} caracteres únicos")
print(f"\n🔤 Caracteres en el vocabulario:")
print(chars_vocab)

In [ ]:
# Crear diccionarios de mapeo caracter <-> índice
char2idx = {ch: idx for idx, ch in enumerate(chars_vocab)}
idx2char = {idx: ch for ch, idx in char2idx.items()}

# Mostrar ejemplos de mapeo
print("🔗 Ejemplos de mapeo char2idx:")
for ch in ['a', 'e', 'i', 'o', 'u', ' ', '.', ',', 'ñ']:
    if ch in char2idx:
        print(f"  '{ch}' -> {char2idx[ch]}")

In [ ]:
# Tokenizar el corpus completo (convertir caracteres a índices)
tokenized_corpus = np.array([char2idx[ch] for ch in corpus], dtype=np.int64)

print(f"📊 Corpus tokenizado - shape: {tokenized_corpus.shape}")
print(f"\n🔢 Primeros 50 tokens:")
print(tokenized_corpus[:50])
print(f"\n📜 Texto correspondiente:")
print(f"'{corpus[:50]}'")

In [ ]:
---
## 4. Estructuración del Dataset

### 4.1 Definición del Tamaño de Contexto

El tamaño de contexto define cuántos caracteres previos utilizará el modelo para predecir el siguiente. En modelos por caracteres podemos usar contextos más largos.

In [ ]:
# Definir tamaño de contexto (hiperparámetro)
MAX_CONTEXT_SIZE = 100

print(f"⚙️ Tamaño de contexto: {MAX_CONTEXT_SIZE} caracteres")
print(f"📝 Esto equivale aproximadamente a {MAX_CONTEXT_SIZE // 5} palabras (asumiendo ~5 caracteres por palabra)")

In [ ]:
### 4.2 División en Entrenamiento y Validación

Dividiremos el corpus secuencialmente: **90% para entrenamiento** y **10% para validación**.

# Proporción para validación
p_val = 0.1

# Calcular índice de división
split_idx = int(len(tokenized_corpus) * (1 - p_val))

# Dividir corpus tokenizado
train_corpus = tokenized_corpus[:split_idx]
val_corpus = tokenized_corpus[split_idx:]

print(f"📊 División del corpus:")
print(f"  • Entrenamiento: {len(train_corpus):,} caracteres ({len(train_corpus)/len(tokenized_corpus)*100:.1f}%)")
print(f"  • Validación: {len(val_corpus):,} caracteres ({len(val_corpus)/len(tokenized_corpus)*100:.1f}%)")

In [ ]:
### 4.3 Creación de Secuencias de Entrenamiento

Estructuramos el problema como **many-to-many**:
- **Entrada**: secuencia de tokens $[x_0, x_1, ..., x_N]$
- **Target**: secuencia desplazada $[x_1, x_2, ..., x_{N+1}]$

Esta estructura permite que cada posición contribuya al gradiente, mejorando el aprendizaje.

In [ ]:
def create_sequences(corpus_data, seq_length):
    """
    Crea secuencias de entrada y target para entrenamiento many-to-many.
    
    Args:
        corpus_data: Array de tokens
        seq_length: Longitud de cada secuencia
    
    Returns:
        X: Array de secuencias de entrada (n_sequences, seq_length)
        y: Array de secuencias target (n_sequences, seq_length)
    """
    n_sequences = len(corpus_data) - seq_length
    
    X = np.zeros((n_sequences, seq_length), dtype=np.int64)
    y = np.zeros((n_sequences, seq_length), dtype=np.int64)
    
    for i in range(n_sequences):
        X[i] = corpus_data[i:i + seq_length]
        y[i] = corpus_data[i + 1:i + seq_length + 1]
    
    return X, y

# Crear secuencias de entrenamiento
X_train, y_train = create_sequences(train_corpus, MAX_CONTEXT_SIZE)

print(f"📦 Secuencias de entrenamiento:")
print(f"  • X_train shape: {X_train.shape}")
print(f"  • y_train shape: {y_train.shape}")

# Verificar alineación entre entrada y target
print("🔍 Ejemplo de alineación entrada-target:")
print(f"\n📥 Entrada (X[0]):")
print(''.join([idx2char[idx] for idx in X_train[0]]))
print(f"\n📤 Target (y[0]):")
print(''.join([idx2char[idx] for idx in y_train[0]]))
print("\n(Observa cómo el target está desplazado un carácter a la derecha)")

In [ ]:
# Crear DataLoaders de PyTorch
BATCH_SIZE = 128

train_dataset = TensorDataset(
    torch.tensor(X_train, dtype=torch.long),
    torch.tensor(y_train, dtype=torch.long)
)

train_loader = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True,
    drop_last=True
)

# Crear secuencias y DataLoader de validación
X_val, y_val = create_sequences(val_corpus, MAX_CONTEXT_SIZE)

val_dataset = TensorDataset(
    torch.tensor(X_val, dtype=torch.long),
    torch.tensor(y_val, dtype=torch.long)
)

val_loader = DataLoader(
    val_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False
)

print(f"📦 DataLoaders creados:")
print(f"  • Batches de entrenamiento: {len(train_loader)}")
print(f"  • Batches de validación: {len(val_loader)}")
print(f"  • Tamaño de batch: {BATCH_SIZE}")

In [ ]:
---
## 5. Definición de Arquitecturas RNN

Implementaremos tres arquitecturas basadas en unidades recurrentes:
1. **SimpleRNN** (Celda de Elman): La más básica, puede sufrir de gradientes que desaparecen
2. **LSTM** (Long Short-Term Memory): Mejor para dependencias largas gracias a compuertas
3. **GRU** (Gated Recurrent Unit): Balance entre complejidad y rendimiento

In [ ]:
class CharLanguageModel(nn.Module):
    """
    Modelo de lenguaje a nivel de caracteres con arquitectura RNN configurable.
    Soporta: RNN, LSTM y GRU.
    """
    
    def __init__(self, vocab_size, hidden_size=256, num_layers=2, 
                 rnn_type='lstm', dropout=0.2):
        super().__init__()
        
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn_type = rnn_type.lower()
        
        # Seleccionar tipo de celda recurrente
        rnn_classes = {'rnn': nn.RNN, 'lstm': nn.LSTM, 'gru': nn.GRU}
        
        if self.rnn_type not in rnn_classes:
            raise ValueError(f"rnn_type debe ser 'rnn', 'lstm' o 'gru'")
        
        # Capa recurrente
        self.rnn = rnn_classes[self.rnn_type](
            input_size=vocab_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, x, hidden=None):
        # One-hot encoding
        x_onehot = F.one_hot(x, num_classes=self.vocab_size).float()
        
        # Forward pass RNN
        rnn_out, hidden = self.rnn(x_onehot, hidden)
        
        # Dropout y capa lineal
        rnn_out = self.dropout(rnn_out)
        logits = self.fc(rnn_out)
        
        return logits, hidden
    
    def init_hidden(self, batch_size, device):
        if self.rnn_type == 'lstm':
            return (
                torch.zeros(self.num_layers, batch_size, self.hidden_size, device=device),
                torch.zeros(self.num_layers, batch_size, self.hidden_size, device=device)
            )
        else:
            return torch.zeros(self.num_layers, batch_size, self.hidden_size, device=device)

print("✅ Clase CharLanguageModel definida correctamente")

In [ ]:
# Comparar arquitecturas en términos de parámetros
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print("📊 Comparación de arquitecturas:")
print("=" * 50)
for rnn_type in ['rnn', 'lstm', 'gru']:
    model_temp = CharLanguageModel(
        vocab_size=vocab_size,
        hidden_size=256,
        num_layers=2,
        rnn_type=rnn_type
    )
    n_params = count_parameters(model_temp)
    print(f"  {rnn_type.upper():>5}: {n_params:>10,} parámetros")
print("=" * 50)

In [ ]:
---
## 6. Entrenamiento del Modelo

### 6.1 Funciones de Entrenamiento y Evaluación

def compute_perplexity(model, data_loader, criterion, device):
    """
    Calcula la perplejidad del modelo en un conjunto de datos.
    Perplejidad = exp(loss promedio). Menor perplejidad = mejor modelo.
    """
    model.eval()
    total_loss = 0
    total_tokens = 0
    
    with torch.no_grad():
        for X_batch, y_batch in data_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            
            logits, _ = model(X_batch)
            loss = criterion(logits.view(-1, model.vocab_size), y_batch.view(-1))
            
            total_loss += loss.item() * y_batch.numel()
            total_tokens += y_batch.numel()
    
    avg_loss = total_loss / total_tokens
    perplexity = np.exp(avg_loss)
    
    return perplexity, avg_loss

print("✅ Función compute_perplexity definida")

def train_model(model, train_loader, val_loader, num_epochs=20, lr=0.001, 
                patience=5, device='cpu', clip_grad=5.0):
    """
    Entrena el modelo con early stopping basado en perplejidad de validación.
    """
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters(), lr=lr)
    
    history = {'train_loss': [], 'train_ppl': [], 'val_loss': [], 'val_ppl': []}
    best_val_ppl = float('inf')
    patience_counter = 0
    best_model_state = None
    
    print(f"🚀 Iniciando entrenamiento - {model.rnn_type.upper()}")
    print("=" * 70)
    
    for epoch in range(num_epochs):
        # --- Fase de Entrenamiento ---
        model.train()
        total_loss = 0
        total_tokens = 0
        
        for X_batch, y_batch in train_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            
            optimizer.zero_grad()
            logits, _ = model(X_batch)
            loss = criterion(logits.view(-1, model.vocab_size), y_batch.view(-1))
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip_grad)
            optimizer.step()
            
            total_loss += loss.item() * y_batch.numel()
            total_tokens += y_batch.numel()
        
        train_loss = total_loss / total_tokens
        train_ppl = np.exp(train_loss)
        
        # --- Fase de Validación ---
        val_ppl, val_loss = compute_perplexity(model, val_loader, criterion, device)
        
        history['train_loss'].append(train_loss)
        history['train_ppl'].append(train_ppl)
        history['val_loss'].append(val_loss)
        history['val_ppl'].append(val_ppl)
        
        print(f"Época {epoch+1:2d}/{num_epochs} | Train Loss: {train_loss:.4f} | "
              f"Train PPL: {train_ppl:7.2f} | Val Loss: {val_loss:.4f} | Val PPL: {val_ppl:7.2f}", end='')
        
        if val_ppl < best_val_ppl:
            best_val_ppl = val_ppl
            best_model_state = model.state_dict().copy()
            patience_counter = 0
            print(" ⭐ Mejor!")
        else:
            patience_counter += 1
            print(f" (paciencia: {patience_counter}/{patience})")
            if patience_counter >= patience:
                print(f"\n⏹️ Early stopping en época {epoch+1}")
                break
    
    print("=" * 70)
    print(f"✅ Mejor perplejidad de validación: {best_val_ppl:.2f}")
    
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
    
    return history

print("✅ Función train_model definida")

In [ ]:
### 6.2 Entrenamiento de los Tres Modelos (SimpleRNN, LSTM, GRU)

In [ ]:
# Hiperparámetros de entrenamiento
HIDDEN_SIZE = 256
NUM_LAYERS = 2
DROPOUT = 0.3
LEARNING_RATE = 0.002
NUM_EPOCHS = 25
PATIENCE = 5

# Diccionarios para almacenar modelos e historiales
models = {}
histories = {}

print("⚙️ Hiperparámetros configurados:")
print(f"  • Hidden Size: {HIDDEN_SIZE}")
print(f"  • Num Layers: {NUM_LAYERS}")
print(f"  • Dropout: {DROPOUT}")
print(f"  • Learning Rate: {LEARNING_RATE}")
print(f"  • Max Epochs: {NUM_EPOCHS}")
print(f"  • Patience: {PATIENCE}")

In [ ]:
# Entrenar modelo SimpleRNN
print("\n" + "="*70)
print("📦 ENTRENANDO MODELO: SimpleRNN")
print("="*70 + "\n")

models['rnn'] = CharLanguageModel(
    vocab_size=vocab_size,
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS,
    rnn_type='rnn',
    dropout=DROPOUT
)

histories['rnn'] = train_model(
    models['rnn'], train_loader, val_loader,
    num_epochs=NUM_EPOCHS, lr=LEARNING_RATE,
    patience=PATIENCE, device=device
)

In [ ]:
# Entrenar modelo LSTM
print("\n" + "="*70)
print("📦 ENTRENANDO MODELO: LSTM")
print("="*70 + "\n")

models['lstm'] = CharLanguageModel(
    vocab_size=vocab_size,
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS,
    rnn_type='lstm',
    dropout=DROPOUT
)

histories['lstm'] = train_model(
    models['lstm'], train_loader, val_loader,
    num_epochs=NUM_EPOCHS, lr=LEARNING_RATE,
    patience=PATIENCE, device=device
)

# Entrenar modelo GRU
print("\n" + "="*70)
print("📦 ENTRENANDO MODELO: GRU")
print("="*70 + "\n")

models['gru'] = CharLanguageModel(
    vocab_size=vocab_size,
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS,
    rnn_type='gru',
    dropout=DROPOUT
)

histories['gru'] = train_model(
    models['gru'], train_loader, val_loader,
    num_epochs=NUM_EPOCHS, lr=LEARNING_RATE,
    patience=PATIENCE, device=device
)

In [ ]:
# Graficar curvas de entrenamiento comparativas
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

colors = {'rnn': '#e74c3c', 'lstm': '#3498db', 'gru': '#2ecc71'}
labels = {'rnn': 'SimpleRNN', 'lstm': 'LSTM', 'gru': 'GRU'}

# Perplejidad de validación
for model_type, history in histories.items():
    epochs = range(1, len(history['val_ppl']) + 1)
    axes[0].plot(epochs, history['val_ppl'], color=colors[model_type], 
                 label=labels[model_type], linewidth=2, marker='o', markersize=4)

axes[0].set_xlabel('Época', fontsize=12)
axes[0].set_ylabel('Perplejidad', fontsize=12)
axes[0].set_title('📈 Perplejidad de Validación', fontsize=14)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Loss de entrenamiento y validación
for model_type, history in histories.items():
    epochs = range(1, len(history['train_loss']) + 1)
    axes[1].plot(epochs, history['train_loss'], color=colors[model_type], 
                 label=f"{labels[model_type]} (train)", linewidth=2, linestyle='-')
    axes[1].plot(epochs, history['val_loss'], color=colors[model_type], 
                 label=f"{labels[model_type]} (val)", linewidth=2, linestyle='--')

axes[1].set_xlabel('Época', fontsize=12)
axes[1].set_ylabel('Loss (Cross-Entropy)', fontsize=12)
axes[1].set_title('📉 Curvas de Aprendizaje', fontsize=14)
axes[1].legend(loc='upper right', fontsize=9)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Tabla resumen
print("\n📊 Resumen de Modelos:")
print("=" * 65)
print(f"{'Modelo':<12} {'Parámetros':>15} {'Mejor Val PPL':>15} {'Épocas':>10}")
print("-" * 65)
for model_type in ['rnn', 'lstm', 'gru']:
    n_params = count_parameters(models[model_type])
    best_ppl = min(histories[model_type]['val_ppl'])
    n_epochs = len(histories[model_type]['val_ppl'])
    print(f"{labels[model_type]:<12} {n_params:>15,} {best_ppl:>15.2f} {n_epochs:>10}")
print("=" * 65)

---
## 7. Generación de Secuencias

Implementaremos tres estrategias de generación:
1. **Greedy Search**: Selecciona siempre el carácter más probable (determinístico)
2. **Beam Search Determinístico**: Mantiene los k mejores candidatos
3. **Beam Search Estocástico**: Muestreo con temperatura para diversidad

In [ ]:
# Seleccionar el mejor modelo para generación
best_model_type = min(histories, key=lambda x: min(histories[x]['val_ppl']))
model = models[best_model_type]
print(f"🏆 Usando modelo: {best_model_type.upper()} (mejor perplejidad de validación)")
print(f"   Perplejidad: {min(histories[best_model_type]['val_ppl']):.2f}")

### 7.1 Greedy Search

En Greedy Search, siempre seleccionamos el carácter con mayor probabilidad. Es determinístico pero puede producir secuencias repetitivas.

def greedy_search(model, seed_text, max_length, num_chars, device='cpu'):
    """
    Genera texto usando búsqueda voraz (greedy search).
    
    Args:
        model: Modelo de lenguaje entrenado
        seed_text: Texto inicial (semilla)
        max_length: Tamaño máximo de contexto
        num_chars: Número de caracteres a generar
        device: Dispositivo de cómputo
    
    Returns:
        generated_text: Texto generado (incluyendo semilla)
    """
    model.eval()
    model = model.to(device)
    generated_text = seed_text.lower()
    
    with torch.no_grad():
        for _ in range(num_chars):
            # Tokenizar el texto actual
            tokens = [char2idx.get(ch, 0) for ch in generated_text[-max_length:]]
            
            # Pad si es necesario
            if len(tokens) < max_length:
                tokens = [0] * (max_length - len(tokens)) + tokens
            
            x = torch.tensor([tokens], dtype=torch.long, device=device)
            logits, _ = model(x)
            
            # Seleccionar el carácter más probable (greedy)
            next_char_idx = logits[0, -1, :].argmax().item()
            next_char = idx2char[next_char_idx]
            generated_text += next_char
    
    return generated_text

print("✅ Función greedy_search definida")

In [ ]:
# Ejemplos de generación con Greedy Search
seed_texts = [
    "el señor fogg",
    "pasepartout dijo",
    "la vuelta al mundo"
]

print("="*70)
print("🔍 GREEDY SEARCH - Generación de texto")
print("="*70)

for seed in seed_texts:
    generated = greedy_search(model, seed, MAX_CONTEXT_SIZE, num_chars=150, device=device)
    print(f"\n📝 Semilla: '{seed}'")
    print("-"*50)
    print(generated)
    print()


### 7.2 Muestreo con Temperatura

La temperatura controla la "creatividad" del modelo:
- **T < 1**: Más conservador, secuencias predecibles
- **T = 1**: Distribución original del modelo  
- **T > 1**: Más aleatorio, secuencias diversas/caóticas

In [ ]:
def sample_with_temperature(model, seed_text, max_length, num_chars, 
                            temperature=1.0, device='cpu'):
    """
    Genera texto usando muestreo con temperatura.
    
    Args:
        model: Modelo de lenguaje entrenado
        seed_text: Texto inicial
        max_length: Tamaño máximo de contexto
        num_chars: Número de caracteres a generar
        temperature: Parámetro de temperatura (0 < T)
        device: Dispositivo de cómputo
    
    Returns:
        generated_text: Texto generado
    """
    model.eval()
    model = model.to(device)
    generated_text = seed_text.lower()
    
    with torch.no_grad():
        for _ in range(num_chars):
            tokens = [char2idx.get(ch, 0) for ch in generated_text[-max_length:]]
            
            if len(tokens) < max_length:
                tokens = [0] * (max_length - len(tokens)) + tokens
            
            x = torch.tensor([tokens], dtype=torch.long, device=device)
            logits, _ = model(x)
            
            # Aplicar temperatura
            logits_scaled = logits[0, -1, :] / temperature
            probs = F.softmax(logits_scaled, dim=-1)
            
            # Muestrear de la distribución
            next_char_idx = torch.multinomial(probs, num_samples=1).item()
            next_char = idx2char[next_char_idx]
            generated_text += next_char
    
    return generated_text

print("✅ Función sample_with_temperature definida")

In [ ]:
# Demostrar efecto de la temperatura
seed = "el viaje comenzó"
temperatures = [0.2, 0.5, 0.8, 1.0, 1.2, 1.5]

print("="*70)
print("🌡️ EFECTO DE LA TEMPERATURA EN LA GENERACIÓN")
print(f"📝 Semilla: '{seed}'")
print("="*70)

for temp in temperatures:
    generated = sample_with_temperature(
        model, seed, MAX_CONTEXT_SIZE, 
        num_chars=100, temperature=temp, device=device
    )
    print(f"\n🌡️ Temperatura = {temp}")
    print("-"*50)
    print(generated)

In [ ]:
### 7.3 Beam Search Determinístico

Beam Search mantiene múltiples hipótesis (beams) y expande las más prometedoras. Es más sofisticado que greedy pero sigue siendo determinístico.

def beam_search_deterministic(model, seed_text, max_length, num_chars, 
                              beam_width=5, device='cpu'):
    """
    Genera texto usando beam search determinístico.
    
    Args:
        model: Modelo de lenguaje entrenado
        seed_text: Texto inicial
        max_length: Tamaño máximo de contexto
        num_chars: Número de caracteres a generar
        beam_width: Número de hipótesis a mantener
        device: Dispositivo de cómputo
    
    Returns:
        best_sequence: Mejor secuencia generada
        all_sequences: Lista de todas las secuencias finales con sus scores
    """
    model.eval()
    model = model.to(device)
    seed_text = seed_text.lower()
    
    # Inicializar beams: (texto, log_prob_acumulada)
    beams = [(seed_text, 0.0)]
    
    with torch.no_grad():
        for _ in range(num_chars):
            all_candidates = []
            
            for text, score in beams:
                tokens = [char2idx.get(ch, 0) for ch in text[-max_length:]]
                if len(tokens) < max_length:
                    tokens = [0] * (max_length - len(tokens)) + tokens
                
                x = torch.tensor([tokens], dtype=torch.long, device=device)
                logits, _ = model(x)
                log_probs = F.log_softmax(logits[0, -1, :], dim=-1)
                
                # Obtener top-k candidatos
                top_log_probs, top_indices = torch.topk(log_probs, beam_width)
                
                for log_prob, idx in zip(top_log_probs.cpu().numpy(), 
                                         top_indices.cpu().numpy()):
                    new_text = text + idx2char[idx]
                    new_score = score + log_prob
                    all_candidates.append((new_text, new_score))
            
            # Seleccionar los mejores beam_width candidatos
            all_candidates.sort(key=lambda x: x[1], reverse=True)
            beams = all_candidates[:beam_width]
    
    # Normalizar scores por longitud
    final_sequences = [(text, score / len(text)) for text, score in beams]
    final_sequences.sort(key=lambda x: x[1], reverse=True)
    
    return final_sequences[0][0], final_sequences

print("✅ Función beam_search_deterministic definida")

In [ ]:
# Ejemplos de Beam Search Determinístico
print("="*70)
print("📊 BEAM SEARCH DETERMINÍSTICO")
print("="*70)

seed = "el tren partió de"
beam_widths = [1, 3, 5, 10]

for bw in beam_widths:
    best, all_seqs = beam_search_deterministic(
        model, seed, MAX_CONTEXT_SIZE, 
        num_chars=80, beam_width=bw, device=device
    )
    print(f"\n📊 Beam Width = {bw}")
    print(f"Mejor secuencia:")
    print(best)
    print(f"(Score normalizado: {all_seqs[0][1]:.4f})")

In [ ]:
### 7.4 Beam Search Estocástico

Combina beam search con muestreo, permitiendo exploración más diversa del espacio de secuencias. La temperatura controla el balance entre exploración y explotación.

def beam_search_stochastic(model, seed_text, max_length, num_chars, 
                           beam_width=5, temperature=1.0, device='cpu'):
    """
    Genera texto usando beam search estocástico con temperatura.
    
    Args:
        model: Modelo de lenguaje entrenado
        seed_text: Texto inicial
        max_length: Tamaño máximo de contexto
        num_chars: Número de caracteres a generar
        beam_width: Número de hipótesis a mantener
        temperature: Parámetro de temperatura para muestreo
        device: Dispositivo de cómputo
    
    Returns:
        best_sequence: Mejor secuencia generada
        all_sequences: Lista de todas las secuencias finales con sus scores
    """
    model.eval()
    model = model.to(device)
    seed_text = seed_text.lower()
    
    beams = [(seed_text, 0.0)]
    
    with torch.no_grad():
        for _ in range(num_chars):
            all_candidates = []
            
            for text, score in beams:
                tokens = [char2idx.get(ch, 0) for ch in text[-max_length:]]
                if len(tokens) < max_length:
                    tokens = [0] * (max_length - len(tokens)) + tokens
                
                x = torch.tensor([tokens], dtype=torch.long, device=device)
                logits, _ = model(x)
                
                # Aplicar temperatura
                logits_scaled = logits[0, -1, :] / temperature
                probs = F.softmax(logits_scaled, dim=-1)
                log_probs = torch.log(probs + 1e-10)
                
                # Muestrear beam_width candidatos según la distribución
                sampled_indices = torch.multinomial(probs, num_samples=min(beam_width, len(probs)), 
                                                   replacement=False)
                
                for idx in sampled_indices.cpu().numpy():
                    new_text = text + idx2char[idx]
                    new_score = score + log_probs[idx].item()
                    all_candidates.append((new_text, new_score))
            
            all_candidates.sort(key=lambda x: x[1], reverse=True)
            beams = all_candidates[:beam_width]
    
    final_sequences = [(text, score / len(text)) for text, score in beams]
    final_sequences.sort(key=lambda x: x[1], reverse=True)
    
    return final_sequences[0][0], final_sequences

print("✅ Función beam_search_stochastic definida")

In [ ]:
# Comparar Beam Search Estocástico con diferentes temperaturas
print("="*70)
print("🎯 BEAM SEARCH ESTOCÁSTICO - Efecto de la Temperatura")
print("="*70)

seed = "había una vez"
temperatures = [0.3, 0.6, 1.0, 1.5]
beam_width = 5

for temp in temperatures:
    best, all_seqs = beam_search_stochastic(
        model, seed, MAX_CONTEXT_SIZE,
        num_chars=100, beam_width=beam_width,
        temperature=temp, device=device
    )
    print(f"\n🌡️ Temperatura = {temp} | Beam Width = {beam_width}")
    print("-"*60)
    print(best)

In [ ]:
---
## 8. Comparación de Métodos de Generación

# Comparación lado a lado de todos los métodos
seed = "el detective fix"
num_chars = 120

print("="*70)
print("🔬 COMPARACIÓN DE MÉTODOS DE GENERACIÓN")
print(f"📝 Semilla: '{seed}'")
print(f"📏 Caracteres a generar: {num_chars}")
print("="*70)

# 1. Greedy Search
print("\n📍 GREEDY SEARCH")
print("-"*60)
result_greedy = greedy_search(model, seed, MAX_CONTEXT_SIZE, num_chars, device)
print(result_greedy)

# 2. Muestreo con T=0.5
print(f"\n🎲 MUESTREO CON TEMPERATURA = 0.5")
print("-"*60)
result_sample_05 = sample_with_temperature(model, seed, MAX_CONTEXT_SIZE, num_chars, 0.5, device)
print(result_sample_05)

# 3. Muestreo con T=1.0
print(f"\n🎲 MUESTREO CON TEMPERATURA = 1.0")
print("-"*60)
result_sample_10 = sample_with_temperature(model, seed, MAX_CONTEXT_SIZE, num_chars, 1.0, device)
print(result_sample_10)

# 4. Beam Search Determinístico
print("\n📊 BEAM SEARCH DETERMINÍSTICO (beam_width=5)")
print("-"*60)
result_beam_det, _ = beam_search_deterministic(model, seed, MAX_CONTEXT_SIZE, num_chars, beam_width=5, device=device)
print(result_beam_det)

# 5. Beam Search Estocástico
print("\n🎯 BEAM SEARCH ESTOCÁSTICO (beam_width=5, temp=0.7)")
print("-"*60)
result_beam_sto, _ = beam_search_stochastic(model, seed, MAX_CONTEXT_SIZE, num_chars, beam_width=5, temperature=0.7, device=device)
print(result_beam_sto)

In [ ]:
---
## 9. Análisis y Conclusiones

In [ ]:
# Resumen final
print("="*70)
print("📋 RESUMEN Y CONCLUSIONES")
print("="*70)

conclusions = """
📚 CORPUS Y PRE-PROCESAMIENTO:
   • Utilizamos "La Vuelta al Mundo en 80 Días" de Julio Verne
   • Tokenización a nivel de caracteres (vocabulario pequeño pero flexible)
   • Contexto de {} caracteres para capturar dependencias largas
   • División 90% entrenamiento / 10% validación

🏗️ ARQUITECTURAS EVALUADAS:
   • SimpleRNN: Más simple pero sufre de gradientes que desaparecen
   • LSTM: Mejor para dependencias largas gracias a las compuertas
   • GRU: Balance entre complejidad y rendimiento

📈 MÉTRICAS:
   • Perplejidad como medida principal de calidad del modelo
   • Early stopping basado en perplejidad de validación

🎯 ESTRATEGIAS DE GENERACIÓN:

   1. GREEDY SEARCH:
      • Siempre elige el carácter más probable
      • Determinístico y rápido
      • Puede producir texto repetitivo

   2. MUESTREO CON TEMPERATURA:
      • T < 1: Más conservador/predecible
      • T = 1: Distribución original del modelo
      • T > 1: Más creativo pero potencialmente incoherente

   3. BEAM SEARCH DETERMINÍSTICO:
      • Mantiene múltiples hipótesis
      • Mejor calidad que greedy
      • Sigue siendo determinístico

   4. BEAM SEARCH ESTOCÁSTICO:
      • Combina beam search con muestreo
      • Permite diversidad con control de calidad
      • La temperatura controla el balance exploración/explotación

🔑 OBSERVACIONES CLAVE:
   • Las arquitecturas con compuertas (LSTM/GRU) superan a SimpleRNN
   • La temperatura es crucial para balancear coherencia y creatividad
   • Beam search estocástico ofrece el mejor balance para generación
""".format(MAX_CONTEXT_SIZE)

print(conclusions)

In [ ]:
# Guardar el mejor modelo
torch.save({
    'model_state_dict': model.state_dict(),
    'model_type': best_model_type,
    'vocab_size': vocab_size,
    'hidden_size': HIDDEN_SIZE,
    'num_layers': NUM_LAYERS,
    'char2idx': char2idx,
    'idx2char': idx2char,
    'max_context_size': MAX_CONTEXT_SIZE
}, 'best_char_lm_model.pt')

print("✅ Modelo guardado en 'best_char_lm_model.pt'")

In [ ]:
---
## 10. Generación Interactiva (Opcional)

In [ ]:
# Función interactiva para generar texto
def generate_text_interactive(seed_text, method='sampling', temperature=0.8, 
                              beam_width=5, num_chars=150):
    """
    Función wrapper para generación interactiva.
    
    Args:
        seed_text: Texto inicial
        method: 'greedy', 'sampling', 'beam_det', 'beam_sto'
        temperature: Para muestreo y beam estocástico
        beam_width: Para métodos beam
        num_chars: Caracteres a generar
    """
    if method == 'greedy':
        return greedy_search(model, seed_text, MAX_CONTEXT_SIZE, num_chars, device)
    elif method == 'sampling':
        return sample_with_temperature(model, seed_text, MAX_CONTEXT_SIZE, 
                                       num_chars, temperature, device)
    elif method == 'beam_det':
        result, _ = beam_search_deterministic(model, seed_text, MAX_CONTEXT_SIZE,
                                               num_chars, beam_width, device)
        return result
    elif method == 'beam_sto':
        result, _ = beam_search_stochastic(model, seed_text, MAX_CONTEXT_SIZE,
                                           num_chars, beam_width, temperature, device)
        return result
    else:
        return "Método no reconocido. Use: 'greedy', 'sampling', 'beam_det', 'beam_sto'"

# Ejemplo de uso
print("\n" + "="*70)
print("🎮 GENERACIÓN INTERACTIVA")
print("="*70)

texto_generado = generate_text_interactive(
    seed_text="en aquel momento",
    method='beam_sto',
    temperature=0.7,
    beam_width=5,
    num_chars=200
)

print("\n📜 Texto generado:")
print("-"*50)
print(texto_generado)

---
## Fin del Desafío 3

✅ **Objetivos cumplidos:**
1. Corpus seleccionado y preprocesado
2. Tokenización por caracteres implementada
3. Tres arquitecturas RNN evaluadas (SimpleRNN, LSTM, GRU)
4. Cuatro métodos de generación implementados:
   - Greedy Search
   - Muestreo con Temperatura
   - Beam Search Determinístico
   - Beam Search Estocástico

📊 **Resultados**: Las arquitecturas con compuertas (LSTM/GRU) obtuvieron mejor perplejidad que SimpleRNN. El efecto de la temperatura fue evidente en la diversidad y coherencia del texto generado.
